## Preparation

<a href="https://colab.research.google.com/github/vectice/vectice-examples/blob/master/Notebooks/Vanilla/Tutorial/Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>  

Welcome to this Vectice Quickstart notebook!


Through this notebook, we will be illustrating how to log the following information into Vectice using the Vectice Python library:
- Dataset versions
- Model versions
- Code
- Runs and lineage

For more information on the tutorial, please refer to the "Vectice Tutorial Page" inside the app.



### Install Vectice

In [ ]:
#Install Vectice Python library 
# In this tutorial we will do code versioning using github, we also support gitlab
# and bitbucket: !pip install -q "vectice[github, gitlab, bitbucket]"
!pip install vectice[github]

     |████████████████████████████████| 132 kB 22.9 MB/s 
     |████████████████████████████████| 1.1 MB 50.9 MB/s 
     |████████████████████████████████| 192 kB 55.3 MB/s 
     |████████████████████████████████| 271 kB 55.3 MB/s 
     |████████████████████████████████| 160 kB 70.6 MB/s 
     |████████████████████████████████| 75 kB 3.2 MB/s 


In [1]:
#Verify if Vectice python library was installed
!pip3 show vectice

Name: vectice
Version: 0.23.0
Summary: Vectice Python library
Home-page: https://github.com/vectice/vectice-python
Author: Vectice Inc.
Author-email: sdk@vectice.com
License: Apache License 2.0
Location: c:\users\salah\anaconda3\lib\site-packages
Requires: requests, urllib3, python-dotenv
Required-by: 


## Log in to your Vectice account

In [5]:
#Import the Vectice library
from vectice import Vectice
from vectice.models import JobType
from vectice.entity.model import ModelType
import os
import logging
logging.basicConfig(level=logging.INFO)

# Specify the API endpoint for Vectice.
os.environ['VECTICE_API_ENDPOINT']= "beta.vectice.com"

# To use the Vectice Python library, you first need to authenticate your account using an API key.
# You can generate an API key from the Vectice UI, by going to the "My API Keys" section under your profile's picture
# Copy and paste your API key here
os.environ['VECTICE_API_TOKEN'] = "RDo3N9Zz4.6WgryNO96vMYlXKQ1e7jLRDo3N9Zz45w0a3G2BAxkPVbd8pmqJ"

# Next, you need to specify the tutorial project where you will run this notebook using a 
# "Project Token". You can find the "Project Token" under the "Settings" tab of your project.

# Copy and paste your Project Token here
# autocode = True enables you to track your git changes for your code automatically every time you execute a run (see below).
vectice = Vectice(project_token="jPJenrwVh5aeA2OZDVaA", autocode= True)


INFO:vectice.auth:Vectice: Refreshing token... 
INFO:vectice.auth:Success!
INFO:vectice.auth:Vectice: Validating project token... 
INFO:vectice.auth:The entered token is OK, and allows you to work on the 'test' Project, part of the 'test' Workspace


## Creating datasets and dataset versions

Vectice enables you to create datasets with and without connections.

### Datasets with connections

To create datasets with connections, you can use **vectice.create_dataset_with_connection_id()** or **vectice.create_dataset_with_connection_name()**. You can get the connection name and id of the connections you have in your workspace by calling **vectice.list_connections()**.

In [7]:
vectice.list_connections()

PagedResponse(total=1, list=[ConnectionOutput(id=1244, name='test', status='Connected', type='GoogleStorage')], current_page=Page(index=1, size=20))

In [8]:
vectice.create_dataset_with_connection_name(connection_name='Tutorial GCS',
                                            dataset_name="Dataset 1",
                                            files=["gs://vectice_tutorial/kc_house_data.csv"],
                                            data_properties=[("property", "value"), ("property 2", "value ")]
                                            )

INFO:Vectice:Dataset: Dataset 1 has been successfully created.


5663

You can see in the app that a new dataset with the name "Dataset 1" has been created. You can easily find your dataset in the application by typing its name in the global search.

### Dataset without connections

In [9]:
vectice.create_dataset(dataset_name="Dataset 2",data_properties=[("property", "value"), ("property 2", "value")])

INFO:Vectice:Dataset: Dataset 2 has been successfully created.


5664

### Create a dataset and a dataset version at the same time

When creating a new dataset version, if the parent dataset doesn't exist in the project, a new dataset is created automatically and it will contain the first version we created.

In [10]:
dataset_version = vectice.create_dataset_version().with_parent_name("Dataset 3").with_properties([("key", "prop")])

We can check in the app that a new dataset called "Dataset 3" has been created.

### Attach a dataset version to a run

 A run is an execution of a job. You can think of a job like an experiment grouping runs.

When creating a run we need to specify:

 1) a job name (mandatory): Job names are unique for each project
 
 2) a job type (optional)
 
 3) a run name (optional): The run names are unique for each job

Job names, job types and run names are useful to group and search runs in the Vectice UI.

You can also specify inputs when you start your run and outputs when you end it. The inputs can be code, dataset and model versions and the outputs can be dataset and model versions.


In [11]:
dataset_version = vectice.create_dataset_version().with_parent_name("Dataset 3").with_properties([("key", "prop")])
vectice.create_run(job_name = "My First Job", job_type = JobType.PREPARATION, run_name = "Run 1").with_properties([("run key", "run prop")])
vectice.start_run(inputs = [dataset_version])
vectice.end_run()

10265

### Declare files' metadata when creating a dataset version

Vectice enables you to declare your files' metadata when creating a new dataset version.

#### Declare files metadata manually

You can declare your files metadata when creating a new dataset version by using **.with_metadata()**. To use this method, you need to declare at least the file's name. You can also declare the file's size, type, uri, path, digest, creation date, and update date.

In [12]:
dataset_version = vectice.create_dataset_version().with_parent_name("Dataset 4").with_properties([("key", "property")]).with_metadata(
            name="My_file.txt",
            size=10600,
            type="TextFile",
            uri="gs://Bucket_name/My_file.txt",
            itemCreatedDate="2022-01-11",
            itemUpdatedDate="2022-01-11",
        )
vectice.create_run(job_name = "My second Job", job_type = JobType.PREPARATION, run_name = "Run 1").with_properties([("run property", "run value")])
vectice.start_run(inputs = [dataset_version])
vectice.end_run()

10266

The Vectice library automatically detects if there have been changes to the dataset you are using. If it detects changes in the metadata, it will generate a new version of your dataset automatically. Else, it's going to use the latest version of your dataset.
Let's change the update date of our file and see if Vectice creates a new version of our dataset.

We can see in Vectice that a new dataset version is created for the dataset "Dataset" . You can easily find your dataset by going to the application and typing its name on the global search or from the datasets tab of your project.
You can also search for the run and see that the dataset version has been used as input of this run.

In [13]:
dataset_version = vectice.create_dataset_version().with_parent_name("Dataset 4").with_properties([("key", "property")]).with_metadata(
            name="My_file.txt",
            size=10600,
            type="TextFile",
            uri="gs://Bucket_name/My_file.txt",
            itemCreatedDate="2022-01-11",
            ## Here we change the update date
            itemUpdatedDate="2022-01-28",
        )
vectice.create_run(job_name = "My second Job", job_type = JobType.PREPARATION, run_name = "Run 2").with_properties([("run property", "run value")])
vectice.start_run(inputs = [dataset_version])
vectice.end_run()

10267

We can see in Vectice that a new dataset version is created since the file's metadata has been changed. You can easily find your dataset by going to the application and typing its name on the global search or from the datasets tab of your project.
You can also search for the run and see that the dataset version has been used as input of this run.

#### Declare files' metadata automatically for GCS

You can declare your GCS files metadata when creating a new dataset version by using **vectice.create_gcs_dataset**. To use this method, you need a Google Cloud Service Account key. This is used to authenticate your access to Google Cloud Storage. Secondly, you will need to get the uri of the folder/file you intend to use. This is the uri you get when you select the file/folder in Google Cloud Storage.
For this notebook, we use the service account that can be found in the tutorial page to access Vectice tutorial data on GCS.

In [ ]:
# Upload the Google Cloud Service Account key
from google.colab import files
uploaded = files.upload()


In [16]:
import os 
# Double check the json file name below so that it matches the name of the file that you downloaded.

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "readerKey.json"
#uri = ["bucket_name/folder"]  # Folder level example / 'bucket_name/folder/file.csv' (would be file level)
uri = "vectice_tutorial/kc_house_data_cleaned.csv"

metadataset = vectice.create_gcs_dataset_version(uri=uri).with_parent_name("GCS dataset").with_properties([("property", "1"), ("property 2", "2")])

vectice.create_run(job_name = "My second Job", job_type = JobType.PREPARATION, run_name = "Run 3")
vectice.start_run(inputs=[metadataset])
vectice.end_run()

10268

You can see in the application that a new dataset called "GCS dataset" has been created and you can find it by typing its name in the global search or by going to the Datasets tab of your project. You can also find it in the inputs of the run called "Run 3" which can be found from the global search also or from the Runs tab in your project.



#### Declare files' metadata automatically for BigQuery

You can declare your BigQuery files metadata when creating a new dataset version by using vectice.create_bigquery_dataset. To use this method, you need a Google Cloud Service Account key. This is used to authenticate your access to BigQuery. Secondly, you will need to get the url of the table you intend to use. This is the url you get when you select the table/dataset in BigQuery.

In [ ]:
# Upload the Google Cloud Service Account key
from google.colab import files
uploaded = files.upload()

In [25]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "test.json"

#uri = "https://console.cloud.google.com/bigquery?project=Project_Name&d=Dataset_Name&p=Project_Name&t=Table_Name&page=table"
uri = "https://console.cloud.google.com/bigquery?referrer=search&project=qa-environment-281904&d=healthcareData&p=nodal-unity-277700&t=StrokeDataClean&page=table&ws=!1m5!1m4!4m3!1snodal-unity-277700!2shealthcareData!3sStrokeDataClean"

metadataset = vectice.create_bigquery_dataset_version(uri=uri).with_parent_name("BQ dataset").with_properties([("property 1", "1"), ("property 2", "2")])

vectice.create_run(job_name = "My second Job", job_type = JobType.PREPARATION, run_name = "Run 4")
vectice.start_run(inputs= [metadataset])
vectice.end_run()

10270

You can see in the application that a new dataset called "BQ dataset" has been created and you can find it by typing its name in the global search or by going to the Datasets tab of your project. You can also find it in the inputs of the run called "Run 4" which can be found from the global search also or from the Runs tab in your project.

## Create models and model versions

Vectice enables you to create your models and model versions and log the metrics, hyperparameters and model properties

When creating a model version, if there is a model with the same name as the given model name in your project, a new model version is added to the given model. Else, a new model is created automatically.

In [26]:
Vectice.create_model_version().with_parent_name('Regressor')

ModelVersionArtifact(None,ModelVersion(metrics=None, properties=None, version=None, userDeclaredVersion=None, parentName='Regressor', parentId=None, tags=None, algorithmName=None, status=None, type=None))

You can declare your model metrics by using **.with_metrics(metrics)**, hyperparameters and properties by using **.with_properties(properties)**, type by usnig **.with_type()**, the used algorithme by using **.with_algorithm(Algorithme name)** and model attachments, by using **.with_attachments(Attachments)**, when creating a model version.

We also use **.with_user_version()** for model versioning. You can provide a version name for your model version. An error will be thrown if the given user version already exists anvd if you don't provide a version name, the version name will be generated automatically.


### Attach a model version as input or output of a run

In [40]:
vectice.create_run(job_name = "My second Job", job_type = JobType.TRAINING, run_name = "Run 5")
vectice.start_run(inputs=[dataset_version])

metrics = [('metric', 10), ('metric 2', 100)]
properties = [('property', 'value'), ('property 2', 'value')]

model_version = vectice.create_model_version().with_user_version().with_parent_name("Regressor").with_algorithm("Decision Tree").with_properties(properties).with_metrics(metrics)

vectice.end_run(outputs=[model_version])

10278

You can see in the application that a new model version has been added to the model called Regressor. You can find the model by typing its name in the global search or from the Models tab of your project. The model version can be also found in the Run "Run 5" outputs. The run can be found also from the global search or from the Runs tab of your project.

You can also get the list of your models and their information by calling **vectice.list_models()**

In [41]:
vectice.list_models().list

[ModelOutput(createdDate='2022-02-04T15:23:20.247Z', updatedDate='2022-02-04T15:23:20.289Z', version=1, id=2900, authorId=12, workspaceId=1029, projectId=3484, repository=None, deletedDate=None, name='Regressor', type='OTHER', description=None)]

You can use **vectice.update_model()** to update your model's type and description.

In [43]:
vectice.update_model(model_id = 2900, model_type = ModelType.REGRESSION, description = "This is a regression model")

INFO:Vectice:Model: '2900' has been updated


You can see in the UI that the model's Type and description have been updated.

You can also get all the model versions you created in previous runs, for offline analysis and understanding in more details what's driving the models performance by using **vectice.list_model_versions_dataframe(model_id)**

In [ ]:
vectice.list_model_versions_dataframe(model_id = 2899)

## Create code versions

Vectice enables you to track your source code by creating code versions. This can be done automatically and manually.

### Creating a code version automatically

If you are using your local environment with GIT installed or JupyterLab etc... the code tracking can be automated by setting  autocode=True when creating the Vectice instance.

### Creating a code version manually

You can create a code version manually by using:

1- **vectice.create_code_version_with_github_uri()** for GitHub

2- **vectice.create_code_version_with_gitlab_uri()** for GitLab

3- **vectice.create_code_version_with_bitbucket_uri()** for Bitbucket

In [33]:
## Example for code versioning with Github
code_version = Vectice.create_code_version_with_github_uri("https://github.com/vectice/vectice-examples",
        "Notebooks/Vanilla/Tutorial/Quickstart.ipynb")

vectice.create_run(job_name = "My second Job", job_type = JobType.TRAINING, run_name = "Run 6")
vectice.start_run(inputs=[code_version])
vectice.end_run()

10274

You can see in the Vectice UI that the code version has been attached to the run "Run 6". You can  find the run by typing its name in the global search or in the Runs tab of your project.

## What's next 🚀 ?
You can now do the Vectice tutorial that can be found in the "Vectice Tutorial Page" accessible from your account.